# Supplemental Material: SAI Storylines under GLENS

In [ ]:
# __authors__: Elizabeth Barnes and Patrick Keys
# __date__: Aug 01, 2022

In [ ]:
import importlib as imp
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import datetime
from icecream import ic

import data_processing_glens, plots

import cartopy as ct
import palettable
from matplotlib.colors import ListedColormap
import matplotlib as mpl
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["figure.dpi"] = 150
savefig_dpi = 300
np.warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
import warnings
warnings.filterwarnings("ignore")



FS = 10
plt.rc('text',usetex=True)
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']}) 
plt.rc('savefig',facecolor='white')
plt.rc('axes',facecolor='white')
plt.rc('axes',labelcolor='dimgrey')
plt.rc('axes',labelcolor='dimgrey')
plt.rc('xtick',color='dimgrey')
plt.rc('ytick',color='dimgrey')

map_proj = ct.crs.EqualEarth(central_longitude = 0.)

In [ ]:
TREND_YEARS_GLENS = (2020,2029)
TREND_YEARS_CTRL = (2010,2019)
WARMING_CUTOFF = 0.01

DATA_DIRECTORY = '../glens_detection/data/GLENS/'
FIGURE_DIRECTORY = 'figures/supp_glens/'

## Get the data

In [ ]:
imp.reload(data_processing_glens)
mask = data_processing_glens.get_land_mask(DATA_DIRECTORY + 'landSeaMask.r90x45.nc', var="TN10p")
da_all = data_processing_glens.get_data(DATA_DIRECTORY)
da_control = data_processing_glens.get_control_data(DATA_DIRECTORY)

mask['lat'] = da_all['lat']

In [ ]:
imp.reload(data_processing_glens)
da_all_mean = da_all.mean("member")

# --------------------------------
# Control trends
da_mean_trends_cntrl = data_processing_glens.compute_trends(da_all_mean, TREND_YEARS_CTRL[0], TREND_YEARS_CTRL[1])
ic(da_mean_trends_cntrl.shape)

da_mean_trends_cntrl20 = data_processing_glens.compute_trends(da_all_mean, 2010, 2029)
ic(da_mean_trends_cntrl20.shape)

da_trends_cntrl = data_processing_glens.compute_trends(da_all, TREND_YEARS_CTRL[0], TREND_YEARS_CTRL[1])
ic(da_trends_cntrl.shape)

# --------------------------------
# GLENS trends
da_mean_trends_glens = data_processing_glens.compute_trends(da_all_mean, TREND_YEARS_GLENS[0], TREND_YEARS_GLENS[1])
ic(da_mean_trends_glens.shape)

da_mean_trends_glens35 = data_processing_glens.compute_trends(da_all_mean, TREND_YEARS_GLENS[0], TREND_YEARS_GLENS[0]+34)
ic(da_mean_trends_glens35.shape)

da_trends_glens = data_processing_glens.compute_trends(da_all, TREND_YEARS_GLENS[0], TREND_YEARS_GLENS[1])
ic(da_trends_glens.shape)


In [ ]:
# error('here')

## Figure 1

In [ ]:
imp.reload(plots)
PLOT_MEMBER = 8
MULT_FACTOR = 10
FS = 12
#------------------------
# fig = plt.figure(figsize=(13,3*3))
fig = plt.figure(figsize=(5.0*1.5*2*.8,3.75*1.3*3*.8))
spec = fig.add_gridspec(ncols=4, nrows=3, height_ratios=[.75,1,1], width_ratios=[1,.6,.6,1])
vbound = 0.25*MULT_FACTOR
#------------------------

ax = fig.add_subplot(spec[0,1:3])
global_mean_temp = data_processing_glens.compute_global_mean(da_all) - 273.15
global_mean_temp_control = data_processing_glens.compute_global_mean(da_control) - 273.15

plt.axvspan(2020, 2095, alpha=0.1, color='teal')   

iy = np.where(global_mean_temp_control["year"]>=2010)[0]
plt.plot(global_mean_temp_control["year"][iy]+.5,global_mean_temp_control.T[iy], color='deeppink',linewidth=.5,alpha=.15)
plt.plot(global_mean_temp_control["year"][iy]+.5,np.mean(global_mean_temp_control,0)[iy], color='deeppink',linewidth=4., alpha=.3)

iy = np.where(global_mean_temp["year"]>=2020)[0]
plt.plot(global_mean_temp["year"][iy]+.5,global_mean_temp.T[iy], color='dimgray',linewidth=.5,alpha=.25)
plt.plot(global_mean_temp["year"][iy]+.5,np.mean(global_mean_temp,0)[iy], color='k',linewidth=4., alpha=.75)

plots.format_spines(plt.gca())
ymin, ymax = plt.gca().get_ylim()
plt.xticks(np.arange(2010,2095,10),np.arange(2010,2095,10))
plt.yticks(np.arange(10.0,50.0,1.),np.round(np.arange(10.0,50.0,1.),2))
plt.ylabel('temperature (C)',fontsize=FS)
# plt.xlabel('year')    
plt.xlim(2005,2095)
plt.ylim(14.5,21)

plt.text(2020.5,14.55,'SAI deployment',color="teal",fontsize=FS)
plt.text(2060,17.7,'RCP8.5',color="deeppink", alpha=.5,fontsize=FS)
plt.text(2046,15.25,'GLENS-SAI',color="k",fontsize=FS )
plt.title('(A) Global mean temperature',fontsize=FS*1.5)
#------------------------

for MASK_BOOL in (True,):
    for data_type in ("all","member"):
        for start_year in (2010, 2020):

            if data_type=="member":
                if start_year == 2010:            
                    da_plot = da_trends_cntrl[0,PLOT_MEMBER,:,:].squeeze()  
                    title_text = "(D) Member \#" + str(PLOT_MEMBER+1) + " trends, 2010-2019"
                    specs = (2,0)
                elif start_year == 2020:
                    da_plot = da_trends_glens[0,PLOT_MEMBER,:,:].squeeze()
                    title_text = "(E) Member \#" + str(PLOT_MEMBER+1) + " trends, 2020-2029"               
                    specs = (2,2)
            elif data_type=="all":
                if start_year == 2010:            
                    da_plot = da_mean_trends_cntrl20[0,:,:].squeeze()            
                    title_text = "(B) Ensemble mean trends, " + "2010-2019"  
                    specs = (1,0)
                elif start_year == 2020:
                    da_plot = da_mean_trends_glens35[0,:,:].squeeze()
                    title_text = "(C) Ensemble mean trends, " + "2020-2055"
                    specs = (1,2)


            if MASK_BOOL:
                da_plot_member = da_plot * mask
                da_positive_trends = np.ceil(np.abs(da_plot_member.where(da_plot_member>=WARMING_CUTOFF,0.,drop=False)))
                da_positive_trends = da_positive_trends * mask
            else:
                da_plot_member = da_plot
                da_positive_trends = np.ceil(np.abs(da_plot_member.where(da_plot_member>=WARMING_CUTOFF,0.,drop=False)))        
            frac = data_processing_glens.compute_global_mean( da_positive_trends )

            ax = fig.add_subplot(spec[specs[0],specs[1]:specs[1]+2],projection=map_proj)
            cb,image = plots.drawOnGlobe(ax, 
                              map_proj, 
                              data=da_plot_member*MULT_FACTOR, 
                              lats=da_plot_member["lat"],
                              lons=da_plot_member["lon"],
                              cmap='RdBu_r',
                              vmin= -vbound, 
                              vmax= vbound, 
                              inc=None, 
                              cbarBool=False, 
                              contourMap=[], 
                              contourVals = [], 
                              fastBool=True, 
                              extent='both',
                             )   
            ax.text(0,-60,str(int((100*frac).round())) + '\% of area warming',color='black', fontsize=FS, transform=ct.crs.PlateCarree())

            image.set_clim(-vbound,vbound)
            ax.xaxis.set_visible(False)
            ax.yaxis.set_visible(False) 
            ax.set_title(title_text,fontsize=FS*1.5)

    #https://matplotlib.org/3.5.0/api/_as_gen/matplotlib.pyplot.tight_layout.html#matplotlib.pyplot.tight_layout
    # fig.tight_layout(rect=(0.025, 0.025, 0.95, 0.95))    #tuple (left, bottom, right, top), default: (0, 0, 1, 1); 
    # A rectangle in normalized figure coordinates into which the whole subplots area (including labels) will fit.
    fig.tight_layout(pad=1., h_pad=0., w_pad=-2)

    # set colorbar        
    axs = fig.axes
    bounds = np.round(np.arange(-.2,.29,.1),3)*MULT_FACTOR
    cb = fig.colorbar(image, ax=axs, 
                      shrink=0.3, 
                      ticks=bounds,
                      location='bottom',
                      pad=.025,
                      extend='both',
                      label='degrees C per decade',
                     )
    cb.set_ticklabels(['-2', '-1', '0', '1', '2'],fontsize=FS)
    cb.set_label(label='degrees C per decade',fontsize=FS)
    
    # plt.tight_layout() 
    plt.savefig(FIGURE_DIRECTORY + 'GLENS_four_panel_trends_member' + str(PLOT_MEMBER) + '_ensmean.png',  bbox_inches='tight', dpi=savefig_dpi)
    plt.show()

In [ ]:
# MULT_FACTOR = 10
# vbound = 0.15*MULT_FACTOR
# #------------------------

# for MASK_BOOL in (True,False):
#     for sim_type in ("control", "glens"):
#         fig = plt.figure(figsize=(13,4.5*5))
#         if sim_type == "glens":
#             start_year = TREND_YEARS_GLENS[0]
#             end_year = TREND_YEARS_GLENS[1]  
#             da_plot = da_trends_glens[0,:,:,:].squeeze()        
#         else:
#             start_year = TREND_YEARS_CTRL[0]
#             end_year = TREND_YEARS_CTRL[1]
#             da_plot = da_trends_cntrl[0,:,:,:].squeeze()        
#         for ens in range(0,10):

#             if MASK_BOOL:
#                 da_plot_member = da_plot[ens,:,:] * mask
#                 da_positive_trends = np.ceil(np.abs(da_plot_member.where(da_plot_member>=WARMING_CUTOFF,0.,drop=False)))
#                 da_positive_trends = da_positive_trends * mask
#             else:
#                 da_plot_member = da_plot[ens,:,:]
#                 da_positive_trends = np.ceil(np.abs(da_plot_member.where(da_plot_member>=WARMING_CUTOFF,0.,drop=False)))        
#             frac = data_processing.compute_global_mean( da_positive_trends )

#             ax = fig.add_subplot(5,2,ens+1,projection=map_proj)
#             cb,image = plots.drawOnGlobe(ax, 
#                               map_proj, 
#                               data=da_plot_member*MULT_FACTOR, 
#                               lats=da_plot_member["lat"],
#                               lons=da_plot_member["lon"],
#                               cmap='RdBu_r',
#                               vmin= -vbound, 
#                               vmax= vbound, 
#                               inc=None, 
#                               cbarBool=False, 
#                               contourMap=[], 
#                               contourVals = [], 
#                               fastBool=True, 
#                               extent='both',
#                              )   
#             ax.text(0,-60,str(int((100*frac).round())) + '\% of area warming',color='black', fontsize=8, transform=ct.crs.PlateCarree())

#             ax.set_title('member \#' + str(ens+1) + ', ' + str(start_year) + '-' + str(end_year))

#         # set colorbar        
#         axs = fig.axes
#         bounds = np.round(np.arange(-.2,.29,.1),3)*MULT_FACTOR
#         cb = fig.colorbar(image, ax=axs, 
#                           shrink=0.3, 
#                           ticks=bounds,
#                           location='bottom',
#                           pad=.05,
#                           extend='both',
#                           label='degrees C per decade',
#                          )        
#         cb.set_ticklabels(['-2', '-1', '0', '1', '2'])

#         # plt.tight_layout()
#         plt.savefig(FIGURE_DIRECTORY + 'supp_all_member_trends_' + str(start_year) + '_' + str(end_year) + '_mask_' + str(MASK_BOOL) + '.png',  bbox_inches='tight', dpi=savefig_dpi)
#         # plt.show()
#         plt.close()

# Perceived Failures Frequency

In [ ]:
MASK_BOOL = True

cmap = ListedColormap(palettable.colorbrewer.qualitative.Paired_11.mpl_colors)
cmap = plots.get_qual_cmap()
cmap = palettable.colorbrewer.diverging.Spectral_8_r.mpl_colors
cmap = np.delete(cmap,(0,1,3,6),0)
cmap = ListedColormap(cmap)

fig = plt.figure(figsize=(10,3*2.))

da_plot = da_trends_glens[0,:,:,:].squeeze()
da_plot_cntrl = da_trends_cntrl[0,:,:,:].squeeze()
da_quad_check = None
for iq,quad in enumerate((2,1,3,4)):

    if(quad==1):
        da_quad = xr.where((da_plot_cntrl>=WARMING_CUTOFF) & (da_plot>=WARMING_CUTOFF), 1, 0) 
        text_title = "(B) Continued Warming"
    elif(quad==2):
        da_quad = xr.where((da_plot_cntrl<WARMING_CUTOFF) & (da_plot>=WARMING_CUTOFF), 1, 0) 
        text_title = "(A) Rebound Warming"        
    elif(quad==3):
        da_quad = xr.where((da_plot_cntrl<WARMING_CUTOFF) & (da_plot<WARMING_CUTOFF), 1, 0) 
        text_title = "(C) Stabilization"                
    elif(quad==4):
        da_quad = xr.where((da_plot_cntrl>=WARMING_CUTOFF) & (da_plot<WARMING_CUTOFF), 1, 0) 
        text_title = "(D) Recovery"                
    da_quad = da_quad.mean("member")
    if da_quad_check is None:
        da_quad_check = da_quad
    else:
        da_quad_check = da_quad_check + da_quad
    #---------------------
    if MASK_BOOL:
        da_plot_member = da_quad.to_numpy() * mask.to_numpy()
    else:
        da_plot_member = da_quad

    # da_plot_member = da_quad
    # print(np.shape(da_plot_member))
    
    ax = fig.add_subplot(2,2,iq+1,projection=map_proj)
    cb,p = plots.drawOnGlobe(ax, 
                      map_proj, 
                      data=da_plot_member*100, 
                      lats=da_quad["lat"],
                      lons=da_quad["lon"],
                      cmap=cmap,
                      vmin= -5, 
                      vmax= 105, 
                      inc=None, 
                      cbarBool=False, 
                      contourMap=[], 
                      contourVals = [], 
                      fastBool=True, 
                      extent='both',
                     )   
    p.set_clim(10,90)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    p.cmap.set_over('k')
    p.cmap.set_under('.9')
    # cbar = plt.colorbar(p,ax=ax,label='\% of members',ticks=np.arange(10,90+20,20), extend="both",orientation='horizontal',pad=.025,shrink=.5)
    # cbar.ax.set_xticklabels(np.arange(10,90+20,20)) 
    
    # p.set_clim(-5,105)
    # ax.xaxis.set_visible(False)
    # ax.yaxis.set_visible(False) 
    # plt.colorbar(p,ax=ax,label='\% of members')
    
    ax.set_title(text_title)

#https://matplotlib.org/3.5.0/api/_as_gen/matplotlib.pyplot.tight_layout.html#matplotlib.pyplot.tight_layout
# fig.tight_layout(rect=(0.025, 0.025, 0.95, 0.95))    #tuple (left, bottom, right, top), default: (0, 0, 1, 1); 
# A rectangle in normalized figure coordinates into which the whole subplots area (including labels) will fit.
fig.tight_layout(pad=1.08, h_pad=4., w_pad=.5)
# fig.set_tight_layout(False)    

# # set colorbar 
axs = fig.axes
cb = fig.colorbar(p,
                  # cax=None,
                  #use_gridspec=True,
                  ax=axs, 
                  shrink=0.3, 
                  ticks=np.arange(10,90+20,20),
                  location='bottom',
                  pad=.025,
                  extend='both',
                  label='\% of members',
                 )
cb.set_ticklabels(np.arange(10,90+20,20))
    
    
plt.savefig(FIGURE_DIRECTORY + 'GLENS_fourquadrant_frequencies_' + str(TREND_YEARS_GLENS[0]) + '_' + str(TREND_YEARS_GLENS[1]) + '.png', bbox_inches="tight", dpi=savefig_dpi)
plt.show()

In [ ]:
fig = plt.figure(figsize=(5.0*2*1.5*.9*.8,3.75*1.5*2*.75*.8),)#constrained_layout=True) #gridspec_kw={'width_ratios': [3, 2]})
spec = fig.add_gridspec(ncols=12, nrows=10)

MASK_BOOL = True

#-----------------------------------------------------------------
# PERCEIVED FAILURES
# ax = fig.add_subplot(2,2,1,projection=map_proj)
ax = fig.add_subplot(spec[:5,0:6],projection=map_proj)

cmap = plots.get_qual_cmap()
cmap = palettable.colorbrewer.diverging.Spectral_8_r.mpl_colors
cmap = np.delete(cmap,(0,1,3,6),0)
cmap = ListedColormap(cmap)

da_plot = da_trends_glens[0,:,:,:].squeeze()
da_plot_cntrl = da_trends_cntrl[0,:,:,:].squeeze()

da_quad_1 = xr.where((da_plot_cntrl>=WARMING_CUTOFF) & (da_plot>=WARMING_CUTOFF), 1, 0) 
da_quad_2 = xr.where((da_plot_cntrl<WARMING_CUTOFF) & (da_plot>=WARMING_CUTOFF), 1, 0) 
da_quad_3 = xr.where((da_plot_cntrl<WARMING_CUTOFF) & (da_plot<WARMING_CUTOFF), 1, 0) 
da_quad_4 = xr.where((da_plot_cntrl>=WARMING_CUTOFF) & (da_plot<WARMING_CUTOFF), 1, 0) 
da_quad = (da_quad_1 + da_quad_2).sum("member")/len(da_plot["member"].values)

#---------------------
if MASK_BOOL:
    da_plot_member = da_quad.to_numpy() * mask.to_numpy()
else:
    da_plot_member = da_quad

cb,p = plots.drawOnGlobe(ax, 
                  map_proj, 
                  data=da_plot_member*100, 
                  lats=da_quad["lat"],
                  lons=da_quad["lon"],
                  cmap=cmap,
                  vmin= 10, 
                  vmax= 50, 
                  inc=None, 
                  cbarBool=False, 
                  contourMap=[], 
                  contourVals = [], 
                  fastBool=True, 
                  extent='both',
                 )   
   
p.set_clim(10,50)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
# p.cmap.set_over('k')
p.cmap.set_under('.9')
cbar = plt.colorbar(p,ax=ax,label='\% of members',ticks=np.arange(10,50+10,10), extend="both",orientation='horizontal',pad=.025,shrink=.7)
cbar.ax.set_xticklabels(np.arange(10,50+10,10)) 

ax.set_title('GLENS Probability of perceived failure (2020-2029)')     


#-----------------------------------------------------------------
fig.tight_layout(pad=1.08, h_pad=1, w_pad=-4)
# plt.tight_layout()
plt.savefig(FIGURE_DIRECTORY + 'GLENS_perceived_failure_figure4_' + str(TREND_YEARS_GLENS[0]) + '_' + str(TREND_YEARS_GLENS[1]) + '.png', bbox_inches="tight", dpi=savefig_dpi)
plt.show()